In [111]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC 

from sklearn.metrics import classification_report

# from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss, confusion_matrix
from sklearn.model_selection import train_test_split
# from sklearn.datasets import make_classification

In [16]:
df1=pd.read_csv("./data/2014_Financial_Data.csv")
df2=pd.read_csv("./data/2015_Financial_Data.csv")
df3=pd.read_csv("./data/2016_Financial_Data.csv")
df4=pd.read_csv("./data/2017_Financial_Data.csv")
df5=pd.read_csv("./data/2018_Financial_Data.csv")

In [46]:
df1.head()

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,PRICE_VAR,Class,Beat Spy
0,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,1.449400e+10,...,-0.0187,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,-9.323276,0,0
1,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,1.753823e+08,...,NaN,NaN,NaN,NaN,NaN,1.6484,1.7313,-25.512193,0,0
2,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,2.270000e+09,...,0.0618,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,33.118297,1,1
3,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,2.502180e+08,...,0.0211,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,2.752291,1,1
4,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,2.707700e+09,...,0.0257,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,12.897715,1,1


In [41]:
def better_than_spy(row, spy_var):
    price_var = row['PRICE_VAR']
    if price_var > spy_var:
        return 1
    else:
        return 0

In [42]:
# Drop stock ticker and sector
df1 = df1.drop(df1.columns[0], axis = 1)
df2 = df2.drop(df2.columns[0], axis = 1)
df3 = df3.drop(df3.columns[0], axis = 1)
df4 = df4.drop(df4.columns[0], axis = 1)
df5 = df5.drop(df5.columns[0], axis = 1)

df1.drop(['Sector'], axis=1, inplace=True)
df2.drop(['Sector'], axis=1, inplace=True)
df3.drop(['Sector'], axis=1, inplace=True)
df4.drop(['Sector'], axis=1, inplace=True)
df5.drop(['Sector'], axis=1, inplace=True)

# Rename price var
df1.rename(columns={"2015 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df2.rename(columns={"2016 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df3.rename(columns={"2017 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df4.rename(columns={"2018 PRICE VAR [%]": "PRICE_VAR"},inplace=True)
df5.rename(columns={"2019 PRICE VAR [%]": "PRICE_VAR"},inplace=True)

In [49]:
spy = pd.read_csv("./data/SPY.csv", index_col='Date')

spy2015 = spy.loc[['2015-01-01','2016-01-01']]['Open']
spy2015change = ((spy2015[1] - spy2015[0]) / spy2015[0]) * 100
df1['Beat Spy'] = df1.apply(lambda row: better_than_spy(row, spy2015change), axis=1)

spy2016 = spy.loc[['2016-01-01','2017-01-01']]['Open']
spy2016change = ((spy2016[1] - spy2016[0]) / spy2016[0]) * 100
df2['Beat Spy'] = df2.apply(lambda row: better_than_spy(row, spy2016change), axis=1)

spy2017 = spy.loc[['2017-01-01','2018-01-01']]['Open']
spy2017change = ((spy2017[1] - spy2017[0]) / spy2017[0]) * 100
df3['Beat Spy'] = df3.apply(lambda row: better_than_spy(row, spy2017change), axis=1)

spy2018 = spy.loc[['2018-01-01','2019-01-01']]['Open']
spy2018change = ((spy2018[1] - spy2018[0]) / spy2018[0]) * 100
df4['Beat Spy'] = df4.apply(lambda row: better_than_spy(row, spy2018change), axis=1)

spy2019 = spy.loc[['2019-01-01','2020-01-01']]['Open']
spy2019change = ((spy2019[1] - spy2019[0]) / spy2019[0]) * 100
df5['Beat Spy'] = df5.apply(lambda row: better_than_spy(row, spy2019change), axis=1)

In [51]:
imputer = KNNImputer(n_neighbors=5, weights='distance', metric='nan_euclidean', copy=True)

df1_clean = imputer.fit_transform(df1)
df1_clean = pd.DataFrame(df1_clean)
df1_clean.columns = list(df1)

df2_clean = imputer.fit_transform(df2)
df2_clean = pd.DataFrame(df2_clean)
df2_clean.columns = list(df2)

df3_clean = imputer.fit_transform(df3)
df3_clean = pd.DataFrame(df3_clean)
df3_clean.columns = list(df3)

df4_clean = imputer.fit_transform(df4)
df4_clean = pd.DataFrame(df4_clean)
df4_clean.columns = list(df4)

df5_clean = imputer.fit_transform(df5)
df5_clean = pd.DataFrame(df5_clean)
df5_clean.columns = list(df5)

In [52]:
data = pd.concat([df1_clean, df2_clean, df3_clean, df4_clean, df5_clean])

In [53]:
data_info=pd.DataFrame(data.dtypes).T.rename(index={0:'column type'})
data_info=data_info.append(pd.DataFrame(data.isnull().sum()).T.rename(index={0:'null values (nb)'}))
data_info=data_info.append(pd.DataFrame(data.isnull().sum()/data.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
display(data_info)

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,PRICE_VAR,Class,Beat Spy
column type,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,...,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
null values (nb),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
null values (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
X = data.iloc[:,:-3]
y = data.iloc[:,-1:] # for spy, do -2 for buy/sell
y = np.array([int(i[0]) for i in y.values.tolist()])

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [117]:
transformer = RobustScaler(unit_variance=True).fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

In [118]:
svm = SVC(C=1, gamma='scale', class_weight='balanced')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

SVC(C=1, class_weight='balanced')

In [110]:
svm.get_params()

{'C': 1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

0.5717391304347826


In [119]:
target_names = ['Doesnt Beat SPY', 'Beats SPY']
print(classification_report(y_test, svm.predict(X_test), target_names=target_names))

                 precision    recall  f1-score   support

Doesnt Beat SPY       0.57      0.98      0.72      3126
      Beats SPY       0.43      0.02      0.05      2394

       accuracy                           0.56      5520
      macro avg       0.50      0.50      0.38      5520
   weighted avg       0.51      0.56      0.43      5520

